In [1]:
from skimage import io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
with open("data/datasetnowall.csv", "r") as data:
    laser_db = pd.read_csv(data, delimiter=";").copy()

In [3]:
def uniform_rotation(db):
    unidir_map = []
    counter = 0
    buffer = []
    corr_angle = 0

    for i in db:
        i[2] = np.around((((i[2]+i[6]) + np.pi) % (2*np.pi))-np.pi,decimals=5)
        i[6] = 0
        buffer.append(i)
        if counter == 719:
            if corr_angle != 0:
                for j in range(720):
                    buffer[(j+corr_angle)%720][2]=np.radians((j-360)/2)

            buffer.sort(key= lambda i:i[2])
            unidir_map.append(buffer.copy())
            counter = 0
            buffer = []
        else:
            counter += 1

    return unidir_map

In [4]:
laser_db_unidir = uniform_rotation(laser_db.values)

In [5]:
with open('datasetnowall_unidir.csv', 'w') as f:
    f.write("cnt;time;angle;range;pos_x;pos_y;pos_yaw\n")
    for laser in laser_db_unidir:
        for item in laser:
            f.write("{};{};{};{};{};{};{}\n".format(item[0],item[1],item[2],item[3],item[4],item[5],item[6]))
       